In [1]:
#Import libraries
import numpy as np
import os as os
import pickle as pickle
from cymetric.pointgen.pointgen import PointGenerator
import tensorflow as tf
tfk = tf.keras
from cymetric.models.tfhelper import prepare_tf_basis, train_model

In [2]:
#Set up point generator
n_coords = 5
monomial_powers = 5
monomials = monomial_powers*np.eye(n_coords, dtype=np.int64)
coefficients = np.ones(n_coords)
kmoduli = np.ones(1)
ambient = np.array([n_coords-1])
#print(monomials,coefficients,kmoduli,ambient)
pg = PointGenerator(monomials, coefficients, kmoduli, ambient)

#Generate Points
n_pts = 1000
points = pg.generate_points(n_pts)
#print(f'CY condition satisfied?  -->  {np.all(pg.cy_condition(points) < 1e-10)}')


In [3]:
#Import the pre-trained CYMetric model
cymodel_filepath = '/Users/ed/Documents/Projects/G2/Flows/G2Metric/github/cy_models/test_model.keras'
cymetric_model = tf.keras.models.load_model(cymodel_filepath)
#cymetric_model.summary()

###this imported saved/model converts the complex64 dtype to float32 dtype automatically, may not be a workaround for this?

In [4]:
#Define the coordinate change functions
def CoordChange_C5R10(point, inverse=False):
    ## TO DO: vectorise
    if not inverse:
        return np.concatenate((np.real(point),np.imag(point)))
    else:
        return point[:len(point)//2] + 1j * point[len(point)//2:]
        
def CoordChange_C5L7(point, inverse=False):
    ## TO DO: vectorise
    if not inverse:
        one_idx = np.where(np.isclose(point,complex(1,0)))
        dropped_idx = pg._find_max_dQ_coords([point])
        c3_coords = np.delete(point,[one_idx,dropped_idx])
        return np.concatenate((np.real(c3_coords), np.imag(c3_coords), np.array([theta]))) ## theta will be point.theta
    else:
        z1 = 1+0j
        z2 = point[0] + point[3]*1j
        z3 = point[1] + point[4]*1j
        z4 = point[2] + point[5]*1j
        z5 = np.pow(-(1 + z2**5 + z3**5 + z4**5), 0.2) ## TO DO: generalise to cy_eqn(z1,z2,z3,z4)
        one_idx = point.one_idx ## make attribute
        dropped_idx = pg._find_max_dQ_coords([point]) ## make attribute as this point form wont be asosciated to pg  
        return np.insert(np.array([z1,z2,z3,z4]), [one_idx,dropped_idx], [z1,z5]) * np.exp(1j * point[-1])              


In [69]:
#Define the class for the link points
class LinkPoint:
    def __init__(self, cy_point):
        self.cy_point = cy_point
        self.one_idx = np.where(np.isclose(self.cy_point,complex(1,0)))[0][0] #...find the index which is set to 1 to define the patch
        self.dropped_idx = pg._find_max_dQ_coords([self.cy_point])[0]  #...find the index where the CY eqn is subbed, ## pg is external
        
        self.cy_point_in_S9 = self.cy_point/np.linalg.norm(self.cy_point) 
        self.theta = np.random.uniform(low=0., high=2*np.pi) #...sample a random angle
        self.link_point = self.cy_point_in_S9 * np.exp(1j * self.theta) #...multiple that angle in, the plane origin is the normalised cy point and orientation is implicitly set
        self.dtheta = self.cy_point_in_S9 * 1j * np.exp(1j * self.theta) #...the position vector 90degrees around the S1 is parallel to this d\theta, and cause its on sphere it is normalised already
        self.dtheta_R7 = ###transform this
        
        c3_coords = np.delete(self.link_point,[self.one_idx, self.dropped_idx]) #...produce the local complex CY coordinates
        self.link_point_local = np.concatenate((np.real(c3_coords), np.imag(c3_coords), np.array([self.theta]))) #...generate the link local coordinates
        
        self.holomorphic_volume_form = pg.holomorphic_volume_form(cy_point) ## pg is external
        #self.holomorphic_volume_form_C5 = ###

        self.hermitian_metric_holo_upper = cymetric_model(CoordChange_C5R10(self.cy_point).reshape(1, 10)).numpy().reshape((3,3))
        self.hermitian_metric_holo = np.vstack((np.hstack((np.zeros((3,3)),self.hermitian_metric_holo_upper)),np.hstack((np.conjugate(np.transpose(self.hermitian_metric_holo_upper)),np.zeros((3,3))))))
        ###is the above the correct way to make the hermitian metric in the full holo coords as [[0,h],[h^T,0]]?? think so ?!
        self.hermitian_metric_R6 = ###write functions to do this transform
        self.cy_metric_R6 = np.real(self.hermitian_metric_R6)
        self.kahler_form_R6 = np.imag(self.hermitian_metric_R6)
        self.kahler_form_R10 = #Transform###(self.kahler_form_holo) ###apply a series of transforms to make the kahler form into R10

        ###write these below
        #self.phi_R10 = wedge_within_R10(self.kahler_form_R10, self.dtheta_R10) + np.real(self.holomorphic_volume_form_R10)
        #self.phi_R7 = wedge_R6_with_R1(self.kahler_form_R6, self.dtheta) + np.real(self.holomorphic_volume_form_R6) ###?

        #########
        #write out on paper all the coordinate transformations needed for CY3holomorphic -> CY3_R6 -> then hermitian metric split here (h = g +i\omega) -> (1) LR7 OR (2) CY3_C3 -> C5 -> R10
        #...last arrows doen, need to do first arrows
        ########

    def Transform_CY3holo_CY3(self, tensor):
        Jacobian_CY3holo_CY3 = np.hstack((np.eye(3) - 1j * np.eye(3), np.eye(3) + 1j * np.eye(3)))
        tensor_CY3 = np.einsum('ab,cd,bd->ac', Jacobian_CY3holo_CY3, Jacobian_CY3holo_CY3, tensor) ##TO DO: generalise to any rank input
        return tensor_CY3

    def Transform_CY3_C5(self, tensor):
        Jacobian_CY3_C5 = np.zeros((5,3), dtype=np.complex64)
        other_indices = np.delete(np.array(range(5)),[self.one_idx,self.dropped_idx])
        #Jacobian_CY3_C5[self.one_idx,0] = 0. #...unnecessary as already initialise at 0.
        Jacobian_CY3_C5[other_indices[0],0] = 1
        Jacobian_CY3_C5[other_indices[1],1] = 1
        Jacobian_CY3_C5[other_indices[2],2] = 1
        test_pt_CY3coords = self.cy_point[other_indices]
        Jacobian_CY3_C5[self.dropped_idx,0] = -test_pt_CY3coords[0]**4 / (-1 - test_pt_CY3coords[0]**5 - test_pt_CY3coords[1]**5 - test_pt_CY3coords[2]**5)**(0.8)
        Jacobian_CY3_C5[self.dropped_idx,1] = -test_pt_CY3coords[1]**4 / (-1 - test_pt_CY3coords[0]**5 - test_pt_CY3coords[1]**5 - test_pt_CY3coords[2]**5)**(0.8)
        Jacobian_CY3_C5[self.dropped_idx,2] = -test_pt_CY3coords[2]**4 / (-1 - test_pt_CY3coords[0]**5 - test_pt_CY3coords[1]**5 - test_pt_CY3coords[2]**5)**(0.8)
        tensor_C5 = np.einsum('ab,cd,bd->ac', Jacobian_CY3_C5, Jacobian_CY3_C5, tensor) ##TO DO: generalise to any rank input
        return tensor_C5

    def Transform_C5_R10(self, tensor):
        Jacobian_C5_R10 = np.vstack((np.eye(5), 1j * np.eye(5)))
        tensor_R10 = np.einsum('ab,cd,bd->ac', Jacobian_C5_R10, Jacobian_C5_R10, tensor)
        return tensor_R10

test_point = LinkPoint(points[100])

In [70]:
print(test_point.cy_point)
print(test_point.metric_holo_upper)

[ 1.        -3.45749152e-17j  0.58183509+1.76426451e-02j
 -0.77052679-1.78143864e-01j -0.30194869-7.13649384e-01j
 -0.92996876+4.13124392e-02j]
[[-0.02302095  0.00025862 -0.0315739 ]
 [-0.00422757  0.1830251   0.04268585]
 [-0.00145698  0.0035954   0.14486136]]


In [64]:
###testing for above within class

# Set metric in CY3holo
metric_holo_upper = test_point.cy_metric.reshape((3,3))
metric_holo = np.vstack((np.hstack((np.zeros((3,3)),metric_holo_upper)),np.hstack((-np.transpose(metric_holo_upper),np.zeros((3,3))))))

# Transform metric CY3holo --> CY3
Jacobian_CY3holo_CY3 = np.hstack((np.eye(3) - 1j * np.eye(3), np.eye(3) + 1j * np.eye(3)))
metric_CY3 = np.einsum('ab,cd,bd->ac', Jacobian_CY3holo_CY3, Jacobian_CY3holo_CY3, metric_holo)

# Transform metric CY3 --> C5
Jacobian_CY3_C5 = np.zeros((5,3), dtype=np.complex64)
other_indices = np.delete(np.array(range(5)),[test_point.one_idx,test_point.dropped_idx])
#Jacobian_CY3_C5[test_point.one_idx,0] = 0. #...unnecessary as already initialise at 0.
Jacobian_CY3_C5[other_indices[0],0] = 1
Jacobian_CY3_C5[other_indices[1],1] = 1
Jacobian_CY3_C5[other_indices[2],2] = 1
test_pt_CY3coords = test_point.cy_point[other_indices]
Jacobian_CY3_C5[test_point.dropped_idx,0] = -test_pt_CY3coords[0]**4 / (-1 - test_pt_CY3coords[0]**5 - test_pt_CY3coords[1]**5 - test_pt_CY3coords[2]**5)**(0.8)
Jacobian_CY3_C5[test_point.dropped_idx,1] = -test_pt_CY3coords[1]**4 / (-1 - test_pt_CY3coords[0]**5 - test_pt_CY3coords[1]**5 - test_pt_CY3coords[2]**5)**(0.8)
Jacobian_CY3_C5[test_point.dropped_idx,2] = -test_pt_CY3coords[2]**4 / (-1 - test_pt_CY3coords[0]**5 - test_pt_CY3coords[1]**5 - test_pt_CY3coords[2]**5)**(0.8)
metric_C5 = np.einsum('ab,cd,bd->ac', Jacobian_CY3_C5, Jacobian_CY3_C5, metric_CY3)

# Transform C5 --> R10
Jacobian_C5_R10 = np.vstack((np.eye(5), 1j * np.eye(5)))
metric_R10 = np.einsum('ab,cd,bd->ac', Jacobian_C5_R10, Jacobian_C5_R10, metric_C5)

metric_R10

array([[ 0.00000000e+00+0.00000000e+00j,  1.43552676e-01+0.00000000e+00j,
         3.33936740e-01+0.00000000e+00j,  1.80785137e-01-1.41779699e-01j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+1.43552676e-01j,  0.00000000e+00+3.33936740e-01j,
         1.41779699e-01+1.80785137e-01j,  0.00000000e+00+0.00000000e+00j],
       [-1.43552676e-01+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         3.30009274e-01+0.00000000e+00j,  2.13941507e-01-3.22203653e-02j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00-1.43552676e-01j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+3.30009274e-01j,
         3.22203653e-02+2.13941507e-01j,  0.00000000e+00+0.00000000e+00j],
       [-3.33936740e-01+0.00000000e+00j, -3.30009274e-01+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  8.20754986e-02+2.50981403e-01j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00-3.33936740e-01j,
         0.00000000e+00-3.30009274e-